# 第5章 决策树

可以将决策树看成一个 if-then 规则的集合，决策树的根节点到叶子节点的每一条路径对应一条规则，路径的内部节点的特征对应着规则的条件，叶子节点的类对应着规则的结论

决策树还表示为给定特征条件下类的条件概率分布

基于特征空间划分的类的条件概率模型有无穷多个，我们选择的模型应该不仅对训练数据有很好的拟合，而且对未知数据有很好的预测，使用损失函数确定最优决策树，损失函数通常是正则化的极大似然函数

而从所有可能的决策树中选取最优决策树是 NP 完全问题，所以现实中决策树学习算法通常采用启发式方法

决策树的算法包含特征选择、决策树的生成与决策树的剪枝过程。决策树的生成只考虑局部最优，决策树的剪枝则考虑全局最优。

剪枝是为了让决策树对未知数据有很好的泛化能力

## 特征选择

如果利用一个特征进行分类的结果与随机分类的结果没有很大差别，则称这个特征是没有分类能力的，可以选择扔掉这个特征

### 熵与条件熵

首先给出熵与条件熵的定义

熵（entropy）是表示随机变量不确定性的度量，$X$ 是一个取有限值的离散随机变量，概率分布为

$$
P(X=x_{i}) = p_{i}, i = 1,2,\ldots,n
$$

则随机变量 $X$ 的熵定义为

$$
H(X) = -\sum_{i=1}^{n}p_{i}\log p_{i}
$$

若 $p_{i}=0$，则定义 $0\log 0=0$，通常上式中对数以 2 为底（称为比特 bit）或以 $e$ 为底（称为纳特 nat）

由定义可知，熵只依赖于 $X$ 的分布，而与 $X$ 的取值无关，所以可以将 $X$ 的熵记作 $H(p)$，即

$$
H(p) = H(X) = -\sum_{i=1}^{n}p_{i}\log p_{i}
$$

熵越大，随机变量的不确定性就越大

对于随机变量 $(X, Y)$，其联合概率分布为

$$
P(X=x_{i}, Y=y_{i}) = p_{ij}, i = 1,2,\ldots,n; j = 1,2,\ldots,m
$$

条件熵 $H(Y|X)$ 表示在已知随机变量 $X$ 的条件下随机变量 $Y$ 的不确定性，定义为给定 $X$ 的条件下 $Y$ 的条件概率分布的熵对 $X$ 的数学期望

$$
H(Y|X) = \sum_{i=1}^{n}p_{i}H(Y|X=X_{i})
$$

这里 $p_{i} = P(X=x_{i}), i=1,2,\ldots,n$

当熵和条件熵中的概率由数据统计（特别是极大似然估计）得到时，对应称为经验熵和经验条件熵

### 信息增益

信息增益表示得知特征 $X$ 的信息而使得类 $Y$ 的信息的不确定性减少的程度

定义：特征 $A$ 对训练数据集 $D$ 的信息增益 $g(D,A)$，定义为集合 $D$ 的经验熵 $H(D)$ 与给定特征 $A$ 的条件下 $D$ 的经验条件熵 $H(D|A)$ 之差，即

$$
g(D,A) = H(D) - H(D|A)
$$

其中熵 $H(D)$ 表示对训练数据集 $D$ 进行分类的不确定性

$$
H(D) = -\sum_{k=1}^{K}\frac{|C_{k}|}{|D|}\log_{2}\frac{|C_{k}|}{|D|}
$$

其中 $|D|$ 为训练数据集 $D$ 的样本个数，$|C_{k}|$ 为类别为 $C_{k}$ 的样本个数

如果设特征 $A$ 有 $n$ 个不同的取值 $\{a_{1},a_{2},\ldots,a_{n}\}$，根据 $A$ 不同的取值将 $D$ 划分成 $n$ 个子集 $D_{1},D_{2},\ldots,D_{n}$，$|D_{i}|$ 为 $D_{i}$ 的样本个数

子集 $D_{i}$ 中属于类 

而条件熵 $H(D|A)$ 为 $C_{k}$ 的样本集合为 $D_{ik}$，即 $D_{ik} = D_{i}\cap C_{k}$

$$
H(D|A) = \sum_{i=1}^{n}\frac{|D_{i}|}{|D|}H(D_{i}) = -\sum_{i=1}^{n}\frac{|D_{i}|}{|D|}\sum_{k=1}^{K}\frac{|D_{ik}|}{|D_{i}|}\log_{2}\frac{|D_{ik}|}{|D_{i}|}
$$

信息增益准备的特征选择方法是：对训练数据集（或子集），计算每个特征的信息增益，并比较大小，选择信息增益最大的特征

### 信息增益比

以信息增益作为划分标准，存在偏向于选择取值较多的特征的问题，使用信息增益比可以对这问题进行校正

$$
g_{R}(D,A) = \frac{g(D,A)}{H_{A}(D)}
$$

其中 
$$
H_{A}(D) = -\sum_{i=1}^{n}\frac{|D_{i}|}{|D|}\log_{2}\frac{|D_{i}|}{|D|}
$$
表示训练数据集 $D$ 关于特征 $A$ 的值的熵

## 决策树的生成

### ID3 算法

输入：训练数据集 $D$，特征集 $A$，阈值 $\varepsilon$

输出：决策树 $T$

（1）若 $D$ 中所有实例属于同一类 $C_{k}$，则 $T$ 为单节点树，并将类 $C_{k}$ 作为该节点的类标记，返回 $T$

（2）若 $A = \varnothing$，则 $T$ 为单节点树，并将 $D$ 中实例数最大的类 $C_{k}$ 作为该节点的类标记，返回 $T$

（3）否则，计算 $A$ 中各特征对 $D$ 的信息增益，选择信息增益最大的特征 $A_{g}$

（4）如果 $A_{g}$ 的信息增益小于阈值 $\varepsilon$，则 $T$ 为单节点树，并将 $D$ 中实例数最大的类 $C_{k}$ 作为该节点的类标记，返回 $T$

（5）否则，对 $A_{g}$ 的每一个可能值 $a_{i}$，按 $A_{g} = a_{i}$ 将 $D$ 分割成若干个非空子集 $D_{i}$，作为子节点，递归调用，返回 $T$

### C4.5 算法

C4.5 算法与 ID3 算法相似，只是使用信息增益比代替信息增益。

## 决策树的剪枝

决策树容易对训练数据出现过拟合，解决这个问题是考虑决策树的复杂度，对已生成的决策树进行简化，这个过程称为剪枝

决策树的剪枝往往通过极小化决策树整体的损失函数或代价函数来实现，损失函数定义为

$$
C_{\alpha}(T) = \sum_{t=1}^{|T|}N_{t}H_{t}(T) + \alpha|T|
$$

其中经验熵

$$
H_{t}(T) = -\sum_{k}\frac{N_{tk}}{N_{t}}\log\frac{N_{tk}}{N_{t}}
$$

树 $T$ 的叶节点个数为 $|T|$，$t$ 是 $T$ 的叶节点，有 $N_{t}$ 个样本点，其中 $k$ 类样本点有 $N_{tk}$ 个

$\alpha$ 为参数，如果将损失函数记成

$$
C_{\alpha}(T) = C(T) + \alpha|T|
$$

则 $C(T)$ 表示模型对训练数据的预测误差，即拟合度，$|T|$ 表示模型复杂度，参数 $\alpha$ 负责控制两者之间的影响程度

较大的 $\alpha$ 选择较简单的决策树，较小的 $\alpha$ 选择复杂的拟合度较高的决策树，$\alpha = 0$ 表示只考虑训练数据拟合度

### 剪枝算法

输入：生成算法产生的整棵树 $T$，参数 $\alpha$

输出：修剪后的树 $T_{\alpha}$

（1）计算每个节点的经验熵

（2）递归地从叶节点向上剪枝，设一组叶节点在修剪前后的整体树分别为 $T_{Before}$ 和 $T_{After}$，其对应的损失函数分别是 $C_{\alpha}(T_{Before})$ 和 $C_{\alpha}(T_{After})$，如果 $C_{\alpha}(T_{After}) \le C_{\alpha}(T_{Before})$ 则进行剪枝

（3）重复（2）直到不能继续为止
